# Trace Mouse Movements

### 需安裝

```
pip install pyusb

```

### Windows DLL
[libusb-win32-devel-filter-1.2.6.0.exe](https://sourceforge.net/projects/libusb-win32/files/libusb-win32-releases/1.2.6.0/)  

https://sourceforge.net/projects/libusb-win32/  



### Linux 權限
https://ubuntuforums.org/showthread.php?t=1682084  
https://linuxconfig.org/tutorial-on-how-to-write-basic-udev-rules-in-linux  
```
If anyone is interested.. fixed with udev rule:

cd /etc/udev/rules.d/

nano 70-ch341.rules

SUBSYSTEM=="usb", ATTRS{idVendor}=="1a86", GROUP="dialout"
SUBSYSTEM=="usb", ATTRS{idVendor}=="1a86", MODE="666"

# or 
SUBSYSTEM=="usb", GROUP="dialout", MODE="666"

then reloaded udev rules:

$ sudo udevadm control --reload-rules  
$ sudo udevadm trigger

```

In [1]:
import os, sys

def gen_relative_path(target_path): 
    
    def del_same_parents(target_path, current_path):
        if len(target_path) * len(current_path) > 0:
            if target_path[0] == current_path[0]:
                target_path.pop(0)
                current_path.pop(0)
                del_same_parents(target_path, current_path)
            
    current_path = os.getcwd().replace('\\', '/').split('/')
    target_path = target_path.replace('\\', '/').split('/')
    del_same_parents(target_path, current_path)
    
    return ['..'] * len(current_path) + target_path


def append_source_relative_path(source_relative_path):
    sys.path.append(os.sep.join(source_relative_path))

In [2]:
# paths = [gen_relative_path(p) for p in paths]
# print(paths)

In [3]:
paths = [['..', '..', '..', '..', 'Universal Serial Bus', 'bitbucket', 'github', 'codes'], 
         ['..', '..', '..', '..', '..', 'ORM', 'bitbucket', 'github', 'codes'],
         ['..', 'codes']]

for path in paths:
    append_source_relative_path(path)

In [4]:
%pylab inline  

from universal_serial_bus import * 
from usb_hid.orm.hid111 import HIDdevice 

Populating the interactive namespace from numpy and matplotlib


# Choose devices

In [6]:
print(usb.core.show_devices(verbose = False))

DEVICE ID 8087:0a2a on Bus 001 Address 002, Wireless Controller
DEVICE ID 0bda:2838 on Bus 001 Address 007, Specified at interface
DEVICE ID 0bda:2838 on Bus 001 Address 005, Specified at interface
DEVICE ID 046d:c52b on Bus 001 Address 001, Specified at interface
DEVICE ID 04f2:b57e on Bus 001 Address 003, Miscellaneous



In [7]:
import usb

list(usb.core.find(find_all = True))

[<DEVICE ID 8087:0a2a on Bus 001 Address 002>,
 <DEVICE ID 0bda:2838 on Bus 001 Address 007>,
 <DEVICE ID 0bda:2838 on Bus 001 Address 005>,
 <DEVICE ID 046d:c52b on Bus 001 Address 001>,
 <DEVICE ID 04f2:b57e on Bus 001 Address 003>]

In [ ]:
_, devices = find_all_devices_by_class(DEVICE_CLASS.Human_Interface_Device)
devices

In [ ]:
# vendor_id, product_id = 0x4d9, 0x2011  # Filco KB
# vendor_id, product_id = 0x1b3f, 0x2008 # Sound Card  
# vendor_id, product_id = 0x2341, 0x8036 # Arduino Leonardo 
vendor_id, product_id = 1118, 1915     # MS mouse  
vendor_id, product_id = 0x045e, 0x077b     # MS mouse   
 
dev = HIDdevice(vendor_id, product_id)

In [ ]:
def close(self):
    if self.is_open:
        usb.util.dispose_resources(self)
    self.finalize()
        # self.reset()
    
close(dev)

In [ ]:
# dev.close()

In [ ]:
# dev.descriptors_byte_array

In [ ]:
# dev.device_descriptor

In [ ]:
# dev.descriptors_byte_array[dev.device_descriptor[0]:]

In [ ]:
# dev.descriptors_from_config

In [ ]:
dev.descriptors

In [ ]:
dev.descriptors_dbos_enum

In [ ]:
dev.print_descriptors_dbos()

In [ ]:
dev.report_descriptors[0]

In [ ]:
len(dev.report_descriptors[0])

In [ ]:
print(dev.report_descriptor_dbos[0].parse())

## Get Reports

In [ ]:
dev.endpoints

In [ ]:
collected = 0
attempts = 20

ep = dev.endpoints[131]

while collected < attempts :
    try:
        data = ep.read(8)
        collected += 1
        print (data)
        
    except usb.core.USBError as e:
        print(e)

usb.util.release_interface(dev, 0)

In [ ]:
from time import time 

def get_mouse_offset(endpoint, length = 8):    
    try:
        data = ep.read(8)
        t = time()
        x, y = (OrmClassBase.byte_array_to_int(data[i:i+2], byteorder = 'little', signed = True) for i in (2, 4))
        return (t, x, y)
    
    except usb.core.USBError as e:
        return None
    

def trace_mouse(endpoint, attempts = 500, length = 8):    
    collected = 0
    traces = []

    while collected < attempts :
        result = get_mouse_offset(ep, length = 8)
        
        if result is not None: 
            traces.append(result)
            collected += 1

    usb.util.release_interface(ep.device, 0)
    
    return traces

In [ ]:
ep = dev.endpoints[131]

traces = trace_mouse(ep, attempts = 500)
traces[:5]

In [ ]:
traces = np.array(traces)

t = traces[:, 0]
y = traces[:, 2]
y = np.cumsum(y)

plt.plot(t, y);

In [ ]:
dev.close()